In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

final_list=[]

def scrape_dossierBHL(url):

    # Récuperation des données

    # Faites une demande à la page web
    response = requests.get(url)

    # Vérifiez que la requête a réussi
    if response.status_code != 200:
        print(f"Status code: {response.status_code}")
        return None

    # Obtenez le contenu de la page web
    webpage = response.text

    # Créez un objet BeautifulSoup pour parser le HTML
    soup = BeautifulSoup(webpage, "html.parser")

    # Triage des données

    # Obtenir les informations générales
    general_list = soup.find('ul', {'id': 'general-list'})
    li_elements = general_list.find_all('li')

    info = {}
    # Assigner les informations générales dans le pandas 
    for li in li_elements:
        label = li.find('label').text.strip()
        if label in ['Code dossier BHL','Rubrique courte','Sexe','type.canondate','Statut','Statut religieux','Rubrique']:
            value = li.find('span').text.strip() 
            info[label] = value


    # Obtenir les informations de la div avec l'id 'associations-dossierbhl_codebhl'
    div = soup.find('div', {'id': 'associations-dossierbhl_codebhl'})
    # On crée une liste vide pour stocker tous les liens
    links = []
    if div is not None:  # Ajoutez cette vérification
        a_elements = div.find_all('a')


        for a in a_elements:
            links.append(a.text.strip())

    # On ajoute la liste des liens à notre dictionnaire sous la clé "Relations : Dossiers BHL / Codes BHL"
    info["Relations : Dossiers BHL / Codes BHL"] = links


    # Obtenir les informations de la div avec l'id 'associations-saint_dossierbhl'
    div = soup.find('div', {'id': 'associations-saint_dossierbhl'})
            # On crée une liste vide pour stocker tous les liens
    links = []
    if div is not None:  # Ajoutez cette vérification
        a_elements = div.find_all('a')
        for a in a_elements:
            links.append(a.text.strip())

    # On ajoute la liste des liens à notre dictionnaire sous la clé "Relations : Dossiers BHL / Codes BHL"
    info["Relations : Dossiers BHL / Saints"] = links

    return info


def scrape_page(url):
    # Faites une demande à la page web
    response = requests.get(url)

    # Vérifiez que la requête a réussi
    if response.status_code != 200:
        print(f"Status code: {response.status_code}")
        return None

    # Obtenez le contenu de la page web
    webpage = response.text

    # Créez un objet BeautifulSoup pour parser le HTML
    soup = BeautifulSoup(webpage, "html.parser")
    # Trouver la table
    table = soup.find('table', {'class': 'table table-striped table-hover'})

    # Trouver tous les liens dans la table
    links = table.find_all('a')

    # Extraire et imprimer le texte de chaque lien
    for link in links:
        if link.text != '\n\n':
            print(link['href'])
            data = scrape_dossierBHL("https://legendiers-latins.irht.cnrs.fr/"+link['href'])
            final_list.append(data)
            print(data)
    return 

for page in range(1,68):
    url = "https://legendiers-latins.irht.cnrs.fr/legendiers-latins/list/objects/dossierbhl?pageNum="+str(page)+"&type=id"
    scrape_page(url)8**
    print(final_list)


In [4]:
# Créer un DataFrame à partir de la liste de dictionnaires
df = pd.DataFrame(final_list)

df.to_csv('dossier-bhl.csv', index=False)

print(df)

   Code dossier BHL                   Rubrique courte   Sexe          Statut  \
0          Walterus               Walterus de Birbaco  homme            laïc   
1             Wando          Wando ab. Fontanellensis  homme  ecclésiastique   
2     Wandregisilus  Wandregisilus ab. Fontanellensis  homme  ecclésiastique   
3          Waningus                          Waningus  homme  ecclésiastique   
4           Warinus          Warinus ep. Praenestinus  homme  ecclésiastique   
..              ...                               ...    ...             ...   
74        Zosimus01                      Zosimus papa  homme  ecclésiastique   
75        Zosimus02           Zosimus ep. Syracusanus  homme  ecclésiastique   
76            Zotas                          Zotas m.  homme            laïc   
77        Zoticus01  Zoticus et soc. mm. sub Hadriano  homme            laïc   
78        Zoticus02     Zoticus et soc. mm. sub Decio  homme            laïc   

   Statut religieux                    